In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

# Konfiguration des neuronalen Netzes

In [3]:
class NeuronalNet(nn.Module):
        def __init__(self):
            super(NeuronalNet, self).__init__()
            self.lin1 = nn.Linear(8,30)
            self.lin2 = nn.Linear(30,30)
            self.lin3 = nn.Linear(30,1)
        def forward(self,x):
            x = self.lin1(x)
            x = F.relu(x)
            x = F.relu(self.lin2(x))
            x = F.relu(self.lin3(x))
            return x

# Datenbeschaffung

In [3]:
#Pfade der einzelnen Benutzer
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/topTeamsEngland10Jahre.csv
#'C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\topTeamsEngland.csv'

In [5]:
topTeams_england = pd.read_csv('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\topTeamsEngland.csv', sep=';')

In [6]:
df = pd.DataFrame(topTeams_england, columns = ['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR'])

In [7]:
df_names = df.drop_duplicates(['HomeTeam','AwayTeam'],keep = 'last')

In [8]:
df_names = df_names[['HomeTeam','AwayTeam']]

In [9]:
gameList = []
for row in df_names.itertuples():
    data = [[row.HomeTeam,row.AwayTeam]]
    gameList.append(pd.DataFrame(data))

In [4]:
#Die Tore der einzelnen Mannschaften pro Spiel aus den matches in eine Liste schreiben
def inputValues(gameList, df_stats,inputList):
    for match in gameList:
        homeTeam = match.iat[0,0]
        awayTeam = match.iat[0,1]
        input_values= df_stats[df_stats['HomeTeam']==homeTeam]
        input_values= input_values[input_values['AwayTeam']==awayTeam]
        input_values= input_values.reset_index(drop=True)
        input_values = input_values[:-1]
        inputList.append(input_values.loc[:,['FTHG','FTAG']])

In [11]:
unnormaleListe = []
inputValues(gameList, df, unnormaleListe)
print(unnormaleListe[0])

   FTHG  FTAG
0     0     0
1     0     0
2     1     0
3     0     2
4     1     1
5     2     2
6     2     1
7     2     2
8     0     3


In [12]:
#Methode um aus einer Liste eine Tensorliste mit Floatwerten generieren
def dfToTensor(dfList, tensorList):
    for df in dfList:
        tensor = torch.FloatTensor(df.values).view(8)
        tensorList.append(tensor)

In [13]:
input_tensors = []
dfToTensor(unnormaleListe, input_tensors)

RuntimeError: shape '[8]' is invalid for input of size 18

In [5]:
#Outputvalues generieren, indem bei Heimsieg 0, bei unentschieden 1 und bei Auswärtssieg 2 gesetzt wird
def outputValues(gameList, df_stats,outputList):
    for match in gameList:
        homeTeam = match.iat[0,0]
        awayTeam = match.iat[0,1]
        output_values= df_stats[df_stats['HomeTeam']==homeTeam]
        output_values= output_values[output_values['AwayTeam']==awayTeam]
        output_values= output_values.reset_index(drop=True)
        output_values = output_values.tail(1)
        output_values = output_values.loc[:,['FTHG','FTAG']]
        homegoals = output_values.loc[:,['FTHG']].iloc[0]['FTHG']
        awaygoals = output_values.loc[:,['FTAG']].iloc[0]['FTAG']
        if homegoals > awaygoals:
            df_out=[0]
        if homegoals == awaygoals:
            df_out=[1]
        if homegoals < awaygoals:
            df_out=[2]
        outputList.append(df_out)

In [15]:
output_dfList = []
outputValues(gameList, df, output_dfList)

In [6]:
def dfToTensor_forOutput(dfList, tensorList):
    for df in dfList:
        tensor = torch.FloatTensor(df).view(1)
        tensorList.append(tensor)

In [212]:
torch.FloatTensor([2]).view(1).long()

tensor([2])

In [17]:
output_tensors = []
dfToTensor_forOutput(output_dfList, output_tensors)

# Input mehrere zu einem

In [18]:
#Pfade Fabian
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/09_10-13_14.csv
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/10_11-14_15.csv
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/11_12-15_16.csv
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/12_13-16_17.csv
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/13_14-17_18.csv
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/14_15-18_19.csv

#C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\09_10-13_14.csv
#C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\10_11-14_15.csv
#C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\11_12-15_16.csv
#C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\12_13-16_17.csv
#C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\13_14-17_18.csv
#C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\14_15-18_19.csv

In [7]:
def new_inputs(path,TestInputListe,TestOutputListe):
    topTeams = pd.read_csv(path, sep=';')
    df = pd.DataFrame(topTeams, columns = ['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR'])
    df_names = df.drop_duplicates(['HomeTeam','AwayTeam'],keep = 'last')
    df_names = df_names[['HomeTeam','AwayTeam']]
    gameListe = []
    for row in df_names.itertuples():
        data = [[row.HomeTeam,row.AwayTeam]]
        gameListe.append(pd.DataFrame(data))
    inputValues(gameListe, df,TestInputListe)
    outputValues(gameListe,df,TestOutputListe)

In [8]:
inputList_0910_1314 = []
outputList_0910_1314 = []
new_inputs('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\09_10-13_14.csv',inputList_0910_1314,outputList_0910_1314)

inputList_1011_1415 = []
outputList_1011_1415 = []
new_inputs('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\10_11-14_15.csv',inputList_1011_1415,outputList_1011_1415)

inputList_1112_1516 = []
outputList_1112_1516 = []
new_inputs('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\11_12-15_16.csv',inputList_1112_1516,outputList_1112_1516)

inputList_1213_1617 = []
outputList_1213_1617 = []
new_inputs('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\12_13-16_17.csv',inputList_1213_1617,outputList_1213_1617)

inputList_1314_1718 = []
outputList_1314_1718 = []
new_inputs('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\13_14-17_18.csv',inputList_1314_1718,outputList_1314_1718)

inputList_1415_1819 = []
outputList_1415_1819 = []
new_inputs('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\14_15-18_19.csv',inputList_1415_1819,outputList_1415_1819)

In [9]:
#In Liste inputList_1415_1819 waren leere Dataframes drin, die werden in diesem Zwischenschritt rausgelöscht
for i in range(len(inputList_1415_1819)):
    if inputList_1415_1819[i].empty:
        print(i)

11
14
32
46
54
63
85
92
97
109
116


In [10]:
inputList_1415_1819.pop(11)
inputList_1415_1819.pop(13)
inputList_1415_1819.pop(30)
inputList_1415_1819.pop(43)
inputList_1415_1819.pop(50)
inputList_1415_1819.pop(58)
inputList_1415_1819.pop(79)
inputList_1415_1819.pop(85)
inputList_1415_1819.pop(89)
inputList_1415_1819.pop(100)
inputList_1415_1819.pop(106)

outputList_1415_1819.pop(11)
outputList_1415_1819.pop(13)
outputList_1415_1819.pop(30)
outputList_1415_1819.pop(43)
outputList_1415_1819.pop(50)
outputList_1415_1819.pop(58)
outputList_1415_1819.pop(79)
outputList_1415_1819.pop(85)
outputList_1415_1819.pop(89)
outputList_1415_1819.pop(100)
outputList_1415_1819.pop(106)

print(len(inputList_1415_1819))
print(len(outputList_1415_1819))

110
110


In [13]:
tensor_input = []
dfToTensor(inputList_0910_1314,tensor_input)
dfToTensor(inputList_1011_1415,tensor_input)
dfToTensor(inputList_1112_1516,tensor_input)
dfToTensor(inputList_1213_1617,tensor_input)
dfToTensor(inputList_1314_1718,tensor_input)
dfToTensor(inputList_1415_1819,tensor_input)

In [14]:
tensor_output = []
dfToTensor_forOutput(outputList_0910_1314,tensor_output)
dfToTensor_forOutput(outputList_1011_1415,tensor_output)
dfToTensor_forOutput(outputList_1112_1516,tensor_output)
dfToTensor_forOutput(outputList_1213_1617,tensor_output)
dfToTensor_forOutput(outputList_1314_1718,tensor_output)
dfToTensor_forOutput(outputList_1415_1819,tensor_output)

In [15]:
#Dieser Schritt ist relativ unnötig, falls man jedoch alle einzeln als Tensoren braucht
input_tensor_0910_1314 = []
output_tensor_0910_1314 = []
dfToTensor(inputList_0910_1314, input_tensor_0910_1314)
dfToTensor_forOutput(outputList_0910_1314,output_tensor_0910_1314)

input_tensor_1011_1415 = []
output_tensor_1011_1415 = []
dfToTensor(inputList_1011_1415, input_tensor_1011_1415)
dfToTensor_forOutput(outputList_1011_1415,output_tensor_1011_1415)

input_tensor_1112_1516 = []
output_tensor_1112_1516 = []
dfToTensor(inputList_1112_1516, input_tensor_1112_1516)
dfToTensor_forOutput(outputList_1112_1516,output_tensor_1112_1516)

input_tensor_1213_1617 = []
output_tensor_1213_1617 = []
dfToTensor(inputList_1213_1617, input_tensor_1213_1617)
dfToTensor_forOutput(outputList_1213_1617,output_tensor_1213_1617)

input_tensor_1314_1718 = []
output_tensor_1314_1718 = []
dfToTensor(inputList_1314_1718, input_tensor_1314_1718)
dfToTensor_forOutput(outputList_1314_1718,output_tensor_1314_1718)

input_tensor_1415_1819 = []
output_tensor_1415_1819 = []
dfToTensor(inputList_1415_1819, input_tensor_1415_1819)
dfToTensor_forOutput(outputList_1415_1819,output_tensor_1415_1819)

In [16]:
print(len(tensor_input))
print(len(tensor_output))

820
820


# Trainieren des neuronalen Netzes

In [17]:
net = NeuronalNet()

In [18]:
criterion = nn.L1Loss()

In [19]:
optimizer = optim.SGD(net.parameters(),lr =0.001, momentum = 0.9)

In [297]:
testout= net(tensor_input[5])
testout.view(1,1)

tensor([[0.]], grad_fn=<ViewBackward>)

In [298]:
tensor_input[5]

tensor([0., 1., 2., 2., 2., 1., 0., 1.])

In [304]:
tensor_output[5].long()

tensor([2])

In [24]:
for epoch in range(5):
    running_loss = 0.0
    for i in range(len(tensor_output)):
        optimizer.zero_grad()
        outputs = net(tensor_input[i])
        loss = criterion(outputs, tensor_output[i])
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 50 == 49:
            print (epoch + 1 , i+1 , running_loss / 50)
            running_loss = 0.0

1 50 0.84
1 100 0.82
1 150 1.12
1 200 0.78
1 250 0.7
1 300 0.84
1 350 0.88
1 400 0.68
1 450 0.86
1 500 0.82
1 550 0.86
1 600 0.72
1 650 0.76
1 700 0.92
1 750 0.96
1 800 0.78
2 50 0.84
2 100 0.82
2 150 1.12
2 200 0.78
2 250 0.7
2 300 0.84
2 350 0.88
2 400 0.68
2 450 0.86
2 500 0.82
2 550 0.86
2 600 0.72
2 650 0.76
2 700 0.92
2 750 0.96
2 800 0.78
3 50 0.84
3 100 0.82
3 150 1.12
3 200 0.78
3 250 0.7
3 300 0.84
3 350 0.88
3 400 0.68
3 450 0.86
3 500 0.82
3 550 0.86
3 600 0.72
3 650 0.76
3 700 0.92
3 750 0.96
3 800 0.78
4 50 0.84
4 100 0.82
4 150 1.12
4 200 0.78
4 250 0.7
4 300 0.84
4 350 0.88
4 400 0.68
4 450 0.86
4 500 0.82
4 550 0.86
4 600 0.72
4 650 0.76
4 700 0.92
4 750 0.96
4 800 0.78
5 50 0.84
5 100 0.82
5 150 1.12
5 200 0.78
5 250 0.7
5 300 0.84
5 350 0.88
5 400 0.68
5 450 0.86
5 500 0.82
5 550 0.86
5 600 0.72
5 650 0.76
5 700 0.92
5 750 0.96
5 800 0.78
